# Joining the Aggregated Data Together 
## Author: Dulan Wijeratne 1181873

In this notebook we will join the previously aggregated data together. 

We will start by starting a spark session and then reading in the data.

In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/30 18:50:35 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/30 18:50:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/30 18:50:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
merchant_agg = spark.read.parquet("../../../data/insights/agg_insight_data/merchant_agg.parquet/")
consumers_agg = spark.read.parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet/")
orders_agg = spark.read.parquet("../../../data/insights/agg_insight_data/orders_agg.parquet/")
postcode_agg = spark.read.parquet("../../../data/insights/agg_insight_data/postcode_agg.parquet/")
descriptions_agg = spark.read.parquet("../../../data/insights/agg_insight_data/descriptions_agg.parquet/")

Next we want to check for duplicate or unnecessary

In [ ]:
print(merchant_agg.columns)

In [ ]:
print(consumers_agg.columns)

In [7]:
print(orders_agg.columns)

['merchant_abn', 'number_of_orders', 'average_cost_of_order', 'average_spend_per_consumer', 'average_monthly_diff_consumers', 'consumer_diff_over_period', 'average_growth_consumers', 'merchant_revenue_rounded', 'first_recorded_transaction', 'last_recorded_transaction', 'transcation_period_months']


In [ ]:
print(postcode_agg.columns)

In [4]:
postcode_agg = postcode_agg.drop("name")

In [ ]:
print(descriptions_agg.columns)

In [5]:
descriptions_agg = descriptions_agg.drop("merchant_description")

### Joining the Data

In [6]:
joined_df = merchant_agg.join(consumers_agg, on = "merchant_abn", how = "inner").\
                join(orders_agg, on = "merchant_abn", how = "inner").\
                join(postcode_agg, on = "merchant_abn", how = "inner").\
                join(descriptions_agg, on = "merchant_abn", how = "inner")

In [10]:
joined_df = joined_df.fillna(0)

### Saving Data

In [11]:
joined_df.write.mode("overwrite").parquet("../../../data/insights/joined.parquet")

In [ ]:
# number of postcodes / total number of postcodes = reach

# number of consumers / number of repeat consumers = customer retainability

# average number of repeat consumer transaction per merchant.

# avg spend amount per consumer

# estimated consumer growth rate (group by month and calculate average increase / decrease)

# estimated revenue growth rate (group by month and calculate average increase / decrease)



In [12]:
spark.stop()